In [1]:
import sys
sys.path.append("..") ## resetting the path to the parent directory

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [6]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
batch_size = 32

In [9]:
train_generator = train_datagen.flow_from_directory(
        '../data/train_folder',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 10254 images belonging to 12 classes.


In [10]:
validation_generator = test_datagen.flow_from_directory(
        '../data/validation_folder',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

Found 1140 images belonging to 12 classes.


In [11]:
from tensorflow.keras import backend as K
img_width, img_height = 224, 224
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [12]:
#prepared_images = preprocess_input(X_train)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense

In [14]:
base_model = VGG16(include_top=False, weights='imagenet',input_shape = (224,224,3),pooling=max)
base_model.trainable = False

In [15]:
inpt = Input(shape=(224,224,3),name = 'image_input')

In [16]:
output = base_model(inpt)

In [17]:
flat1 = Flatten(name='flatten')(output)
class1 = Dense(1024, activation='relu', name='fc1')(flat1)
output = Dense(12, activation='softmax', name='predictions')(class1)
# define new model
vgg_model = Model(inputs=inpt, outputs=output)

In [18]:
vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              25691136  
_________________________________________________________________
predictions (Dense)          (None, 12)                12300     
Total params: 40,418,124
Trainable params: 25,703,436
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
sgd = SGD(lr=0.001)

In [20]:
vgg_model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
steps_per_epoch = 100
epoch  = 500
def my_gen(gen):
    i=0
    while i < steps_per_epoch * epoch:
        try:
            data, labels = next(gen)
            i+=1
            yield data, labels
        except:
            pass

In [22]:
hist= vgg_model.fit(
        my_gen(train_generator),
        steps_per_epoch=steps_per_epoch,
        epochs=epoch,
        validation_data=validation_generator)

Epoch 1/1000
100/100 [==============================] - 77s 773ms/step - loss: 2.4267 - accuracy: 0.1391 - val_loss: 2.2895 - val_accuracy: 0.1974
Epoch 2/1000
100/100 [==============================] - 77s 766ms/step - loss: 2.2099 - accuracy: 0.2422 - val_loss: 2.1299 - val_accuracy: 0.2474
Epoch 3/1000
100/100 [==============================] - 78s 775ms/step - loss: 2.0938 - accuracy: 0.2747 - val_loss: 1.9978 - val_accuracy: 0.3009
Epoch 4/1000
100/100 [==============================] - 77s 766ms/step - loss: 1.9797 - accuracy: 0.3174 - val_loss: 1.9143 - val_accuracy: 0.3298
Epoch 5/1000
100/100 [==============================] - 77s 771ms/step - loss: 1.9182 - accuracy: 0.3434 - val_loss: 1.8346 - val_accuracy: 0.3737
Epoch 6/1000
100/100 [==============================] - 77s 772ms/step - loss: 1.8415 - accuracy: 0.3625 - val_loss: 1.7455 - val_accuracy: 0.3956
Epoch 7/1000
100/100 [==============================] - 77s 768ms/step - loss: 1.7969 - accuracy: 0.3843 - val_loss: 1

KeyboardInterrupt: 

In [ ]:
#vgg_history = vgg_model.fit(X_train, y_train, epochs=10,steps_per_epoch=50,validation_data=(X_val,y_val))

In [ ]:
# test_loss, test_acc = vgg_model.evaluate(X_test, y_test, verbose=0)

# print('\nTest accuracy {:5.2f}%'.format(100*test_acc))

In [ ]:
training_loss = vgg_history.history['loss']
test_loss = vgg_history.history['val_loss']

epoch_count = range(1,len(training_loss)+1)

plt.plot(epoch_count,training_loss,'r--')
plt.plot(epoch_count,test_loss,'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
acc = vgg_history.history['accuracy']
val_acc = vgg_history.history['val_accuracy']

epoch_count = range(1,len(training_loss)+1)

plt.plot(acc, 'r--', label='Training Accuracy')
plt.plot(val_acc, 'b-', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.xlabel('Epoch')
plt.show();

## Undersample data